In [34]:
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import json

# Step 1: Load Data from JSON File
with open('new.json', 'r') as file:
    data = json.load(file)

# Convert the data to a pandas DataFrame
df = pd.DataFrame(data)

# Ensure 'text' and 'label' columns are present
assert 'text' in df.columns and 'label' in df.columns, "Data must contain 'text' and 'label' columns"

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Step 2: Load Tokenizer and Model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(df['label'].unique()))

# Step 3: Preprocess the Data
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

encoded_dataset = dataset.map(preprocess_function, batched=True)

# Step 4: Set Up Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01
)

# Step 5: Create Trainer Object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset  # Ideally, use a separate validation set
)

# Step 6: Train the Model
trainer.train()

# Step 7: Save the Model
model.save_pretrained('./model')
tokenizer.save_pretrained('./model')

# Step 8: Make Predictions
from transformers import pipeline

nlp = pipeline('text-classification', model='./model', tokenizer='./model')
text = "Your input text here"
print(nlp(text))


RuntimeError: Failed to import transformers.models.roberta.configuration_roberta because of the following error (look up to see its traceback):
Failed to import transformers.onnx.config because of the following error (look up to see its traceback):
cannot import name '_imaging' from 'PIL' (/usr/lib/python3/dist-packages/PIL/__init__.py)

In [28]:
pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install pillow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [33]:
pip cache purge

Files removed: 2
Note: you may need to restart the kernel to use updated packages.
